<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

# [AutoKeras](https://autokeras.com/)

** Note: This code has been tested as of July 2021**

As AI is automating more and more jobs, it can finally automate designing AI architectures too. Neural Architecture Search (NAS) approaches utilize reinforcement learning to join together mini architectural blocks, till they are able to maximize the objective function - i.e. our validation accuracy. The current state of the art networks are all based on NAS, leaving human-designed architectures in the dust. Research in this area started showing promising results in 2017, with a bigger focus on making train faster in 2018.

AutoKeras (Haifeng Jin et al), also apply this state of the art technique on our particular datasets in a relatively accessible manner. Generating new model architectures with AutoKeras is a matter of supplying our images and associated labels, and a time limit to finish running the jobs by. Internally, it implements several optimization algorithms including a Bayesian optimization approach to search for an optimal architecture.

In [2]:
!pip install tensorflow
!pip install autokeras
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 613.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.6/924.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 38.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: ten

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak

Using TensorFlow backend


Load the standard MNIST data

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Define an image classifier. The `max_trials` setting can be played around with to enable more experimentation. It refers to the maximum number of different Keras Models to try. The search may finish before reaching the max_trials. Defaults to 100.

In [5]:
clf = ak.ImageClassifier(overwrite=True, max_trials=10)

Fit the classifier.

In [6]:
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 03m 30s]
val_loss: 0.07086892426013947

Best val_loss So Far: 0.07086892426013947
Total elapsed time: 00h 03m 30s
1875/1875 [==============================] - 161s 86ms/step - loss: 0.1532 - accuracy: 0.9535


Prediction based on the best model trained

In [7]:
predicted_y = clf.predict(x_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_true=y_test, y_pred=predicted_y.flatten().astype("uint8")))

313/313 [==============================] - 6s 19ms/step
0.9829


Evaluate the best model with testing data

In [8]:
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 10s 31ms/step - loss: 0.0493 - accuracy: 0.9829
[0.04934513568878174, 0.9829000234603882]


Export as a Keras Model

In [9]:
model = clf.export_model()

Let's visualize the trained model.

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 cast_to_float32 (CastToFlo  (None, 28, 28)            0         
 at32)                                                           
                                                                 
 expand_last_dim (ExpandLas  (None, 28, 28, 1)         0         
 tDim)                                                           
                                                                 
 normalization (Normalizati  (None, 28, 28, 1)         3         
 on)                                                             
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                             

In [11]:
print(type(model))

<class 'keras.src.engine.functional.Functional'>


Now, let's write the model to disk

In [12]:
# Save the model to load it in the What-If tool
model_path = "model_autokeras"
if IS_COLAB_ENV:
    model_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-5/{model_path}"

try:
    model.save(model_path, save_format="tf")
except:
    model.save(f"{model_path}.h5")

Let's read the model we just wrote.

In [13]:
from tensorflow.keras.models import load_model

loaded_model = load_model(model_path, custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))

313/313 [==============================] - 6s 18ms/step
